# Generate ATP Synthase with Proton Pump SBML files
Will try to incorporate it instead of using SBML 

---

In [2]:
from biocrnpyler import *
#from biocrnpyler.chemical_reaction_network import Species, Reaction, ComplexSpecies, Multimer
import numpy as np
import pylab as plt 
%matplotlib inline

import bokeh.io
import bokeh.plotting
bokeh.io.output_notebook()

import colorcet

Loading BokehJS ...

ATP synthase DNA --> ATP synthase RNA <br>
ATP synthase RNA --> ATP synthase protein (folding) <br>
ATP synthase protein --> ATP synthase membrane integration <br>
<br> 
ATP synthesis through ATP synthase protein <br>
ATP hydrolysis through ATP synthase protein <br>
Proton transport through ATP synthase protein <br>
<br>
Proton pump DNA --> Proton pump RNA <br>
Proton pump RNA --> Proton pump protein (folding) <br>
Proton pump protein --> Proton pump membrane integration <br>
<br>
Proton transport through proton pump

**SBML Planning** <br>
- One sbml file for ATP synthase membrane integration
- One sbml file for ATP synthase synthesis/hydrolysis/proton transport 
- One sbml file for proton pump membrane integration
- One sbml file for proton pump proton transport

## SBML for ATP transcription and translation
---

In [52]:

#promoter is the name of the promoter or a Promoter component
#RBS is the name of the RBS or an RBS Component
#   RBS = None means there will be no translation
#By default (if transcript and protein are None), the transgript and protein will have the same name as an assmebly
#   Users can also assign Species or String names to transcript and protein
G = DNAassembly("atpsyn", promoter = "atpsyn_promoter", rbs = "atpsyn_rbs", transcript = None, protein = None)
M3 = TxTlDilutionMixture("liposome", components = [G], parameter_file = "default_parameters.txt")
CRN3 = M3.compile_crn()


print(repr(M3),"\n", CRN3.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")


TxTlDilutionMixture: liposome
Components = [
	DNAassembly: atpsyn
	Protein: RNAP
	Protein: Ribo
	Protein: RNAase
	DNAassembly: cellular_processes ]
Mechanisms = {
	transcription:transcription_mm
	translation:translation_mm
	rna_degredation:rna_degredation_mm
	catalysis:michalis_menten
	binding:one_step_binding }
Global Mechanisms = {
	dilution:global_degredation_via_dilution } 
 Species (15) = {0. complex[dna[atpsyn]:protein[RNAP](machinery)], 1. rna[atpsyn], 2. complex[protein[RNAase]:rna[atpsyn](machinery)], 3. dna[atpsyn], 4. protein[RNAP(machinery)], 5. complex[protein[Ribo]:rna[atpsyn](machinery)], 6. protein[atpsyn], 7. protein[RNAase(machinery)], 8. protein[Ribo(machinery)], 9. dna[cellular_processes], 10. complex[protein[RNAase]:rna[cellular_processes](machinery)], 11. protein[cellular_processes], 12. complex[protein[Ribo]:rna[cellular_processes](machinery)], 13. complex[dna[cellular_processes]:protein[RNAP](machinery)], 14. rna[cellular_processes]}
Reactions (16) = [
0. dna[at

In [20]:
# Write SBML file
CRN3.write_sbml_file('atpsynthase_tx_tl.xml')

True

In [21]:
timepoints = np.linspace(0,10000,100)
x0 = {'dna_atpsyn':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R = CRN3.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [22]:
#R

In [3]:
palette = colorcet.b_glasbey_category10
palette.remove('#d62628')

In [26]:
p = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase tx and tl')
plot_list = ['rna_atpsyn', 'protein_atpsyn', 'dna_atpsyn']
for i,item in enumerate(plot_list):
    p.line(timepoints, R[item], legend_label = item, color = palette[i], line_width = 2)
    
p.legend.location = 'center_right'
bokeh.io.show(p)

## SBML 2 (simple) for ATP Synthase Tx and Tl
---

In [65]:
rna_atpsyn = Species("atpsyn", material_type = "rna")

dna_atpsyn = Species("atpsyn", material_type = "dna")

f_atpsyn = Species("atpsyn", material_type = "protein")

k_tx = 1
k_tl = 1

R = Reaction([dna_atpsyn],[rna_atpsyn], k_tx)

R1 = Reaction([rna_atpsyn], [f_atpsyn], k_tl)
CRN_simpletxtl = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn], reactions = [R, R1])

print(CRN_simpletxtl.pretty_print(show_materials = True, show_rates = True, show_attributes = True))


Species (3) = {0. rna[atpsyn], 1. dna[atpsyn], 2. protein[atpsyn]}
Reactions (2) = [
0. dna[atpsyn] --> rna[atpsyn]        
        massaction: k_f(dna[atpsyn])=1*dna[atpsyn]
1. rna[atpsyn] --> protein[atpsyn]        
        massaction: k_f(rna[atpsyn])=1*rna[atpsyn]
]


In [66]:
# Write SBML file
CRN_simpletxtl.write_sbml_file('atpsynthase_simple_txtl.xml')

True

In [34]:
x0 = {'dna_atpsyn' : 10}
timepoints = np.linspace(0,100,100)
r_simple = CRN_simpletxtl.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

#results, _ = CRN_machinerytxtl.simulate_via_bioscrape(timepoints, initial_condition_dict = x0, sbml_warnings = False)

In [43]:
p_simple = bokeh.plotting.figure(width = 450, height = 350, title = 'ATP Synthase with Machinery',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = [ 'dna_atpsyn']
#plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'free atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_simple.line(timepoints, r_simple[item], line_width = 2, legend_label = item, color = palette[i])
    
bokeh.io.show(p_simple)

## SBML 3 (manual RNA poly, Ribo, mrna degradation) for ATP Synthase Tx and Tl
---

In [5]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap_atpsyn = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo_atpsyn = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna_atpsyn = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap_atpsyn], k = kb) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap_atpsyn], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo_atpsyn], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo_atpsyn],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna_atpsyn], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna_atpsyn], [RNAse_machinery], k = k5)

CRN_machinerytxtl = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, 
                                                       complex_dna_rnap_atpsyn, complex_rna_ribo_atpsyn, 
                                     complex_rnase_rna_atpsyn],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_machinerytxtl.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[atpsyn], 1. dna[atpsyn], 2. protein[atpsyn], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[atpsyn]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[atpsyn]], 8. complex[protein[RNAse_machinery]:rna[atpsyn]]}
Reactions (6) = [
0. dna[atpsyn] + protein[RNAP_machinery] --> complex[dna[atpsyn]:protein[RNAP_machinery]]        
        massaction: k_f(dna[atpsyn],protein[RNAP_machinery])=100.0*dna[atpsyn]*protein[RNAP_machinery]
1. complex[dna[atpsyn]:protein[RNAP_machinery]] --> rna[atpsyn] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[atpsyn]:protein[RNAP_machinery]])=0.05*complex[dna[atpsyn]:protein[RNAP_machinery]]
2. rna[atpsyn] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[atpsyn]]        
        massaction: k_f(rna[atpsyn],protein[Ribo_machinery])=100.0*rna[atpsyn]*protein[Ribo_machinery]
3. complex[protein[Ribo_machinery]:rna[atpsyn]] --> rna[atpsyn

In [6]:
# Write SBML file
CRN_machinerytxtl.write_sbml_file('atpsynthase_machinery_txtl.xml')

True

In [7]:
x0 = {'dna_atpsyn' : 10, 'protein_RNAP_machinery' : 10, 'protein_Ribo_machinery' : 10, 'protein_RNAse_machinery' : 10}
timepoints = np.linspace(0,1000,100)
r = CRN_machinerytxtl.simulate_with_bioscrape(timepoints, initial_condition_dict = x0)

#results, _ = CRN_machinerytxtl.simulate_via_bioscrape(timepoints, initial_condition_dict = x0, sbml_warnings = False)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [8]:
p_mach = bokeh.plotting.figure(width = 450, height = 350, title = 'ATP Synthase with Machinery',
                          x_axis_label = 'time',
                          y_axis_label = 'concentration')
plot_list = ['protein_atpsyn','dna_atpsyn', 'rna_atpsyn']
#plot_str = ['atp', 'outside_H', 'inside_H', 'bound atp synthase', 'free atp synthase']
# plot_list = [atp_synthase_b_protein_id, atp_synthase_f_protein_id]
# plot_str = ['atp synthase bound', 'atp synthase free']
for i,item in enumerate(plot_list):
    p_mach.line(timepoints, r[item], line_width = 2, legend_label = item, color = palette[i])
p_mach.legend.click_policy = 'hide'
p_mach.legend.location = 'top_left'
bokeh.io.show(p_mach)

In [65]:
r.head()

,rna_atpsyn,dna_atpsyn,protein_atpsyn,protein_RNAP_machinery,protein_Ribo_machinery,protein_RNAse_machinery,complex_dna_atpsyn_protein_RNAP_machinery,complex_protein_Ribo_machinery_rna_atpsyn,complex_protein_RNAse_machinery_rna_atpsyn,time
0,0.000000,10.000000,0.000000,10.000000,10.000000,10.000000,0.000000,0.000000,0.000000,0.000000
1,0.000261,0.070461,0.006221,0.070461,9.753535,9.747493,9.929539,0.246465,0.252507,1.010101
2,0.000274,0.070461,0.024796,0.070461,9.511893,9.488042,9.929539,0.488107,0.511958,2.020202
3,0.000288,0.070461,0.055429,0.070461,9.275938,9.223169,9.929539,0.724062,0.776831,3.030303
4,0.000302,0.070461,0.097843,0.070461,9.045337,8.953214,9.929539,0.954663,1.046786,4.040404


## SBML for atp synthase temperature sensitive txtl 25 C
---

In [38]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 25 

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_25temp_machinerytxtl_atpsyn = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_25temp_machinerytxtl_atpsyn.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[atpsyn], 1. dna[atpsyn], 2. protein[atpsyn], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[atpsyn]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[atpsyn]], 8. complex[protein[RNAse_machinery]:rna[atpsyn]]}
Reactions (6) = [
0. dna[atpsyn] + protein[RNAP_machinery] --> complex[dna[atpsyn]:protein[RNAP_machinery]]        
        massaction: k_f(dna[atpsyn],protein[RNAP_machinery])=0.0002564102564102564*dna[atpsyn]*protein[RNAP_machinery]
1. complex[dna[atpsyn]:protein[RNAP_machinery]] --> rna[atpsyn] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[atpsyn]:protein[RNAP_machinery]])=0.05*complex[dna[atpsyn]:protein[RNAP_machinery]]
2. rna[atpsyn] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[atpsyn]]        
        massaction: k_f(rna[atpsyn],protein[Ribo_machinery])=100.0*rna[atpsyn]*protein[Ribo_machinery]
3. complex[protein[Ribo_machinery]:rna[atpsyn]

In [39]:
# Write SBML file
CRN_25temp_machinerytxtl_atpsyn.write_sbml_file('atpsyn_25temp_machinery_txtl.xml')

True

## SBML for atp synthase temperature sensitive txtl 37 C
---

In [50]:
rna_atpsyn = Species("atpsyn", material_type = "rna")
dna_atpsyn = Species("atpsyn", material_type = "dna")
f_atpsyn = Species("atpsyn", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_atpsyn, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_atpsyn, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_atpsyn, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 100

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_atpsyn, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_atpsyn, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_atpsyn, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_atpsyn, f_atpsyn, Ribo_machinery], k = k1)
R4 = Reaction([rna_atpsyn, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_100temp_machinerytxtl_atpsyn = ChemicalReactionNetwork(species = [rna_atpsyn, dna_atpsyn, f_atpsyn, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_100temp_machinerytxtl_atpsyn.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[atpsyn], 1. dna[atpsyn], 2. protein[atpsyn], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[atpsyn]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[atpsyn]], 8. complex[protein[RNAse_machinery]:rna[atpsyn]]}
Reactions (6) = [
0. dna[atpsyn] + protein[RNAP_machinery] --> complex[dna[atpsyn]:protein[RNAP_machinery]]        
        massaction: k_f(dna[atpsyn],protein[RNAP_machinery])=4.3859649122807014e-05*dna[atpsyn]*protein[RNAP_machinery]
1. complex[dna[atpsyn]:protein[RNAP_machinery]] --> rna[atpsyn] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[atpsyn]:protein[RNAP_machinery]])=0.05*complex[dna[atpsyn]:protein[RNAP_machinery]]
2. rna[atpsyn] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[atpsyn]]        
        massaction: k_f(rna[atpsyn],protein[Ribo_machinery])=100.0*rna[atpsyn]*protein[Ribo_machinery]
3. complex[protein[Ribo_machinery]:rna[atpsyn

In [51]:
# Write SBML file
CRN_100temp_machinerytxtl_atpsyn.write_sbml_file('atpsyn_100temp_machinery_txtl.xml')

True

## SBML for ATP synthase membrane integration
---

In [9]:
atpsyn = Species("atpsyn", material_type = "protein")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

k_bound = 0.0003

R = Reaction([atpsyn], [atpsyn_b], k_bound)

#R1 = Reaction([atpsyn], [], k_bound)

CRN_mi = ChemicalReactionNetwork(species = [atpsyn, atpsyn_b], reactions = [R])

print(CRN_mi.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (2) = {0. protein[atpsyn], 1. protein[atpsyn_b]}
Reactions (1) = [
0. protein[atpsyn] --> protein[atpsyn_b]        
        massaction: k_f(protein[atpsyn])=0.0003*protein[atpsyn]
]


In [10]:
# Write SBML file
CRN_mi.write_sbml_file('atpsynthase_membrane_integration.xml')

True

In [11]:
timepoints = np.linspace(0,100,100)
x0_mi = {'protein_atpsyn':10}
R_mi = CRN_mi.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi)

In [12]:
R_mi

,protein_atpsyn,protein_atpsyn_b,time
0,10.000000,0.000000,0.000000
1,9.996970,0.003030,1.010101
2,9.993941,0.006059,2.020202
3,9.990913,0.009087,3.030303
4,9.987886,0.012114,4.040404
...,...,...,...
95,9.716225,0.283775,95.959596
96,9.713282,0.286718,96.969697
97,9.710339,0.289661,97.979798
98,9.707397,0.292603,98.989899


In [13]:
p_mi = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthase membrane integration')
plot_list = ['protein_atpsyn', 'protein_atpsyn_b']
for i,item in enumerate(plot_list):
    p_mi.line(timepoints, R_mi[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi.legend.location = 'center_right'
bokeh.io.show(p_mi)

## SBML for ATP Synthesis via ATP synthase membrane protein
---

In [11]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

atpsyn_b = Species("atpsyn_b", material_type = "protein")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 50

R1 = Reaction([outside_H, outside_H, outside_H, outside_H,adp, pi, atpsyn_b], 
              [inside_H, inside_H, inside_H, inside_H,atp, atpsyn_b], k_atp_synthesis)

# R3 = Reaction([inside_H, inside_H, inside_H, inside_H, atpsyn_b],
#               [outside_H, outside_H, outside_H, outside_H, atpsyn_b], k_atp_synthesis)

R2 = Reaction([adp, pi], [atp], k_atp_synthesis)

# R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
#              [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, atpsyn_b, adp, atp, pi], reactions = [R1])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (6) = {0. proton[outside_H], 1. proton[inside_H], 2. protein[atpsyn_b], 3. metabolite[adp], 4. metabolite[atp], 5. metabolite[pi]}
Reactions (1) = [
0. 4 proton[outside_H] + metabolite[adp] + metabolite[pi] + protein[atpsyn_b] --> 4 proton[inside_H] + metabolite[atp] + protein[atpsyn_b]        
        massaction: k_f(proton[outside_H],metabolite[adp],metabolite[pi],protein[atpsyn_b])=50*proton[outside_H]^4*metabolite[adp]*metabolite[pi]*protein[atpsyn_b]
]


In [12]:
# Write SBML file
CRN_atpsynthesis.write_sbml_file('atp_synthesis_through_atpsynthase_try2.xml')

True

In [24]:
timepoints = np.linspace(0,2,100)
x0_atpsynthesis = {'proton_outside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_atpsynthesis = CRN_atpsynthesis.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpsynthesis)

In [25]:
p_atpsynthesis = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP synthesis through ATP synthase')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpsynthesis.line(timepoints, R_atpsynthesis[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpsynthesis.legend.location = 'center_right'
bokeh.io.show(p_atpsynthesis)

## SBML for Proton Pump transcription and translation
---

In [46]:
G_pp = DNAassembly("proton_pump", promoter = "atpsyn_proton_pump", rbs = "proton_pumnp_rbs", transcript = None, protein = None)
M3_pp = TxTlDilutionMixture("liposome", components = [G_pp], parameter_file = "default_parameters.txt")
CRN3_pp = M3_pp.compile_crn()

#print(repr(M3),"\n", CRN3_pp.pretty_print(show_attributes = True, show_material = True, show_rates = True),"\n\n")



In [47]:
# Write SBML file
CRN3_pp.write_sbml_file('proton_pump_tx_tl.xml')

True

In [49]:
timepoints = np.linspace(0,10000,100)
x0_pp = {'dna_proton_pump':10, 'protein_RNAP_machinery':10, 'protein_Ribo_machinery':10,
     'protein_RNAase_machinery':10}
R_pp = CRN3_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_pp)

/Users/ankitaroychoudhury/opt/anaconda3/lib/python3.7/site-packages/scipy/integrate/odepack.py:248: ODEintWarning: Excess work done on this call (perhaps wrong Dfun type). Run with full_output = 1 to get quantitative information.
  warnings.warn(warning_msg, ODEintWarning)
odeint failed with mxstep=500...

In [50]:
p_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump tx and tl')
plot_list = ['rna_proton_pump', 'protein_proton_pump', 'dna_proton_pump']
for i,item in enumerate(plot_list):
    p_pp.line(timepoints, R_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_pp.legend.location = 'center_right'
bokeh.io.show(p_pp)

## SBML for proton pump Simple tx tl
---

In [77]:
rna_proton_pump = Species("proton_pump", material_type = "rna")

dna_proton_pump = Species("proton_pump", material_type = "dna")

f_proton_pump = Species("proton_pump", material_type = "protein")

k_tx = 1
k_tl = 1

R = Reaction([dna_proton_pump],[rna_proton_pump], k_tx)

R1 = Reaction([rna_proton_pump], [f_proton_pump], k_tl)
CRN_simpletxtl_pp = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump], 
                                         reactions = [R, R1])

print(CRN_simpletxtl_pp.pretty_print(show_materials = True, show_rates = True, show_attributes = True))



Species (3) = {0. rna[proton_pump], 1. dna[proton_pump], 2. protein[proton_pump]}
Reactions (2) = [
0. dna[proton_pump] --> rna[proton_pump]        
        massaction: k_f(dna[proton_pump])=1*dna[proton_pump]
1. rna[proton_pump] --> protein[proton_pump]        
        massaction: k_f(rna[proton_pump])=1*rna[proton_pump]
]


In [78]:
# Write SBML file
CRN_simpletxtl_pp.write_sbml_file('proton_pump_simple_txtl.xml')

True

## SBML for proton pump machinery txtl
---

In [8]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap_pp = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo_pp = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna_pp = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
# A = 0.0576
# B = -4.68
# C = 120
# T = 25 

# #rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

# k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap_pp], k = kb) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap_pp], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo_pp], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo_pp],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna_pp], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna_pp], [RNAse_machinery], k = k5)

CRN_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, 
                                        complex_dna_rnap_pp, complex_rna_ribo_pp, complex_rnase_rna_pp],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[proton_pump], 1. dna[proton_pump], 2. protein[proton_pump], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[proton_pump]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[proton_pump]], 8. complex[protein[RNAse_machinery]:rna[proton_pump]]}
Reactions (6) = [
0. dna[proton_pump] + protein[RNAP_machinery] --> complex[dna[proton_pump]:protein[RNAP_machinery]]        
        massaction: k_f(dna[proton_pump],protein[RNAP_machinery])=100.0*dna[proton_pump]*protein[RNAP_machinery]
1. complex[dna[proton_pump]:protein[RNAP_machinery]] --> rna[proton_pump] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[proton_pump]:protein[RNAP_machinery]])=0.05*complex[dna[proton_pump]:protein[RNAP_machinery]]
2. rna[proton_pump] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[proton_pump]]        
        massaction: k_f(rna[proton_pump],protein[Ribo_machinery])=100.0*rna[proton_pu

In [93]:
# Write SBML file
CRN_machinerytxtl_proton_pump.write_sbml_file('proton_pump_machinery_txtl.xml')

True

## SBML for proton pump temperature sensitive txtl 25 C
---

In [35]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 25 

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_25temp_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_25temp_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[proton_pump], 1. dna[proton_pump], 2. protein[proton_pump], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[proton_pump]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[proton_pump]], 8. complex[protein[RNAse_machinery]:rna[proton_pump]]}
Reactions (6) = [
0. dna[proton_pump] + protein[RNAP_machinery] --> complex[dna[proton_pump]:protein[RNAP_machinery]]        
        massaction: k_f(dna[proton_pump],protein[RNAP_machinery])=0.0002564102564102564*dna[proton_pump]*protein[RNAP_machinery]
1. complex[dna[proton_pump]:protein[RNAP_machinery]] --> rna[proton_pump] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[proton_pump]:protein[RNAP_machinery]])=0.05*complex[dna[proton_pump]:protein[RNAP_machinery]]
2. rna[proton_pump] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[proton_pump]]        
        massaction: k_f(rna[proton_pump],protein[Ribo_machinery])=100

In [36]:
# Write SBML file
CRN_25temp_machinerytxtl_proton_pump.write_sbml_file('proton_pump_25temp_machinery_txtl.xml')

True

## SBML for proton pump temperature sensitive txtl 37 C
---

In [48]:
rna_proton_pump = Species("proton_pump", material_type = "rna")
dna_proton_pump = Species("proton_pump", material_type = "dna")
f_proton_pump = Species("proton_pump", material_type = "protein")

RNAP_machinery = Species('RNAP_machinery', material_type = 'protein')
Ribo_machinery = Species("Ribo_machinery", material_type = 'protein')
RNAse_machinery = Species("RNAse_machinery", material_type = 'protein')

complex_dna_rnap = ComplexSpecies([dna_proton_pump, RNAP_machinery])
complex_rna_ribo = ComplexSpecies([rna_proton_pump, Ribo_machinery])
complex_rnase_rna = ComplexSpecies([rna_proton_pump, RNAse_machinery])

kb = 100.0
kf = 1.0
k1 = 0.05
k5 = 0.001
A = 0.0576
B = -4.68
C = 120
T = 100

#rate_string= "(" + str(dna)+ "*"+ "(" + "1" + "/" + "(" + str(A) + "*" + str(Temperature) + "^" + "2" + "+" + str(B) + "*" + str(Temperature) + "+" + str(C) + ")"+ ")" + ")"

k_temp = 0.01 * (1/(A * T**2 + B * T + C))

R0 = Reaction([dna_proton_pump, RNAP_machinery], [complex_dna_rnap], k = k_temp) # THIS ONE WILL BE TEMP DEPENDENT, 
                                                                        # made irreversible for now
R1 = Reaction([complex_dna_rnap], [rna_proton_pump, RNAP_machinery], k = k1) #dna_atpsyn,
R2 = Reaction([rna_proton_pump, Ribo_machinery], [complex_rna_ribo], k = kb) #k_rev = kf)
R3 = Reaction([complex_rna_ribo],[rna_proton_pump, f_proton_pump, Ribo_machinery], k = k1)
R4 = Reaction([rna_proton_pump, RNAse_machinery], [complex_rnase_rna], k = kb)# k_rev = kf
R5 = Reaction([complex_rnase_rna], [RNAse_machinery], k = k5)

CRN_100temp_machinerytxtl_proton_pump = ChemicalReactionNetwork(species = [rna_proton_pump, dna_proton_pump, f_proton_pump, RNAP_machinery, Ribo_machinery,
                                         RNAse_machinery, complex_dna_rnap, complex_rna_ribo, complex_rnase_rna],
                               reactions = [R0, R1, R2, R3, R4, R5])

print(CRN_100temp_machinerytxtl_proton_pump.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (9) = {0. rna[proton_pump], 1. dna[proton_pump], 2. protein[proton_pump], 3. protein[RNAP_machinery], 4. protein[Ribo_machinery], 5. protein[RNAse_machinery], 6. complex[dna[proton_pump]:protein[RNAP_machinery]], 7. complex[protein[Ribo_machinery]:rna[proton_pump]], 8. complex[protein[RNAse_machinery]:rna[proton_pump]]}
Reactions (6) = [
0. dna[proton_pump] + protein[RNAP_machinery] --> complex[dna[proton_pump]:protein[RNAP_machinery]]        
        massaction: k_f(dna[proton_pump],protein[RNAP_machinery])=4.3859649122807014e-05*dna[proton_pump]*protein[RNAP_machinery]
1. complex[dna[proton_pump]:protein[RNAP_machinery]] --> rna[proton_pump] + protein[RNAP_machinery]        
        massaction: k_f(complex[dna[proton_pump]:protein[RNAP_machinery]])=0.05*complex[dna[proton_pump]:protein[RNAP_machinery]]
2. rna[proton_pump] + protein[Ribo_machinery] --> complex[protein[Ribo_machinery]:rna[proton_pump]]        
        massaction: k_f(rna[proton_pump],protein[Ribo_machinery])=10

In [49]:
# Write SBML file
CRN_100temp_machinerytxtl_proton_pump.write_sbml_file('proton_pump_100temp_machinery_txtl.xml')

True

## SBML for proton pump membrane integration
---

In [23]:
proton_pump = Species("proton_pump", material_type = "protein")

proton_pump_b = Species("proton_pump_b", material_type = "protein")

k_bound = 0.0003

R_pp = Reaction([proton_pump], [proton_pump_b], k_bound)

CRN_mi_pp = ChemicalReactionNetwork(species = [proton_pump, proton_pump_b], reactions = [R_pp])

print(CRN_mi_pp.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (2) = {0. protein[proton_pump], 1. protein[proton_pump_b]}
Reactions (1) = [
0. protein[proton_pump] --> protein[proton_pump_b]        
        massaction: k_f(protein[proton_pump])=0.0003*protein[proton_pump]
]


In [24]:
# Write SBML file
CRN_mi_pp.write_sbml_file('proton_pump_membrane_integration.xml')

True

In [73]:

outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_atp_synthesis = 2

R1 = Reaction([adp,pi], [atp], k_atp_synthesis)

R2 = Reaction([outside_H, outside_H, outside_H, outside_H], 
             [inside_H, inside_H, inside_H, inside_H], k_atp_synthesis)

CRN_atpsynthesis = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1,R2])

print(CRN_atpsynthesis.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (5) = {0. proton[outside_H], 1. proton[inside_H], 2. metabolite[adp], 3. metabolite[atp], 4. metabolite[pi]}
Reactions (2) = [
0. metabolite[adp] + metabolite[pi] --> metabolite[atp]        
        massaction: k_f(metabolite[adp],metabolite[pi])=2*metabolite[adp]*metabolite[pi]
1. 4 proton[outside_H] --> 4 proton[inside_H]        
        massaction: k_f(proton[outside_H])=2*proton[outside_H]^4
]


In [74]:
timepoints = np.linspace(0,10,100)
x0_mi_pp = {'protein_proton_pump':10}
R_mi_pp = CRN_mi_pp.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_mi_pp)

In [75]:
p_mi_pp = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton pump membrane integration')
plot_list = ['protein_proton_pump', 'protein_proton_pump_b']
for i,item in enumerate(plot_list):
    p_mi_pp.line(timepoints, R_mi_pp[item], legend_label = item, color = palette[i], line_width = 2)
    
p_mi_pp.legend.location = 'center_right'
bokeh.io.show(p_mi_pp)

## SBML for proton transport through proton pump
---

In [13]:
outside_H = Species("outside_H", material_type = "proton")

inside_H = Species("inside_H", material_type = "proton")

adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')


k_proton_out = 200
k_proton_atp = 0.2

#R1 = Reaction([adp,pi], [atp], k_proton_atp)
R1 = Reaction([atp], [adp, pi], k_proton_atp)


R2 = Reaction([inside_H, inside_H, inside_H, inside_H],
              [outside_H, outside_H, outside_H, outside_H], k_proton_out)

CRN_protontransport = ChemicalReactionNetwork(species = [outside_H, inside_H, adp, atp, pi], reactions = [R1, R2])

print(CRN_protontransport.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

Species (5) = {0. proton[outside_H], 1. proton[inside_H], 2. metabolite[adp], 3. metabolite[atp], 4. metabolite[pi]}
Reactions (2) = [
0. metabolite[atp] --> metabolite[adp] + metabolite[pi]        
        massaction: k_f(metabolite[atp])=0.2*metabolite[atp]
1. 4 proton[inside_H] --> 4 proton[outside_H]        
        massaction: k_f(proton[inside_H])=200*proton[inside_H]^4
]


In [14]:
# Write SBML file
CRN_protontransport.write_sbml_file('proton_transport_through_proton_pump.xml')

True

In [10]:
timepoints = np.linspace(0,2,100)
x0_protontransport = {'proton_inside_H':10, 'metabolite_adp':10, 'metabolite_pi':10}
R_protontransport = CRN_protontransport.simulate_with_bioscrape(timepoints, 
                                                                initial_condition_dict = x0_protontransport)


In [11]:
p_protontransport = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'Proton transport through proton pump')
plot_list = ['proton_outside_H', 'proton_inside_H', 'metabolite_atp']
for i,item in enumerate(plot_list):
    p_protontransport.line(timepoints, R_protontransport[item], legend_label = item, color = palette[i], line_width = 2)
    
p_protontransport.legend.location = 'center_right'
bokeh.io.show(p_protontransport)

## SBML for ATP Use
---

In [1]:
adp = Species('adp', material_type = 'metabolite')
atp = Species('atp', material_type = 'metabolite')
pi = Species('pi', material_type = 'metabolite')

k_atp_use = 75
# used to be 5

R1_atpuse = Reaction([atp], [adp, pi], k_atp_use)


CRN_atpuse = ChemicalReactionNetwork(species = [adp, atp, pi], reactions = [R1_atpuse])

print(CRN_atpuse.pretty_print(show_materials = True, show_rates = True, show_attributes = True))

NameError: name 'Species' is not defined

In [2]:
# Write SBML file
CRN_atpuse.write_sbml_file('atp_75_use.xml')

NameError: name 'CRN_atpuse' is not defined

In [38]:
timepoints = np.linspace(0,75000,100)
plt_timepoints = timepoints/60/60
x0_atpuse = {'metabolite_atp':10}
R_atpuse = CRN_atpuse.simulate_with_bioscrape(timepoints, initial_condition_dict = x0_atpuse)

In [39]:
p_atpuse = bokeh.plotting.figure(width = 450, height = 340, 
                         x_axis_label = 'time (sec)', y_axis_label = 'concentration',
                         title = 'ATP Use')
plot_list = ['metabolite_atp']
for i,item in enumerate(plot_list):
    p_atpuse.line(plt_timepoints, R_atpuse[item], legend_label = item, color = palette[i], line_width = 2)
    
p_atpuse.legend.location = 'center_right'
bokeh.io.show(p_atpuse)